# Plan and Execute Pattern using Microsoft Agent Framework

This notebook demonstrates the implementation of the **Plan and Execute** pattern using Microsoft Agent Framework. This pattern improves agent performance by:

1. **Breaking down complex tasks** into manageable sub-tasks (Planning)
2. **Executing each sub-task** in sequence or parallel
3. **Adapting to feedback** during execution

## Architecture Overview

The Plan and Execute pattern involves:
- **Planner**: Responsible for generating a structured plan of sub-tasks
- **Executors**: Handle the execution of each sub-task
- **Workflow**: Coordinates sequential or parallel execution
- **Tools**: Custom functions that can be called during execution

![Plan and Execute Pattern](../../images/planning.png)

## What You'll Learn

In this notebook, you will learn how to:

1. **Set up Microsoft Agent Framework** with Azure OpenAI
2. **Create custom tools** for web search and product research
3. **Build sequential workflows** for step-by-step task execution
4. **Build parallel workflows** for concurrent task execution
5. **Implement plan-and-execute patterns** for complex queries

## Prerequisites

- Azure OpenAI endpoint and API key
- Google Search API credentials (optional, for web search)
- Microsoft Agent Framework installed: `pip install agent-framework azure-ai`

## Understanding the Plan and Execute Pattern

The Plan and Execute pattern we'll demonstrate offers several advantages:

1. **Task Decomposition**: Complex tasks are broken down into simpler, manageable steps
2. **Tool Selection**: The workflow automatically routes to the appropriate executors for each step
3. **Adaptability**: If a step fails, the workflow can adapt by trying alternative approaches
4. **Explainability**: The workflow structure provides transparency into how the agent approaches problems

### Pattern Comparison

| Pattern | Use Case | Complexity | Flexibility |
|---------|----------|------------|-------------|
| **Sequential** | Linear multi-step tasks | Low | Limited |
| **Parallel (Fan-out/Fan-in)** | Independent concurrent tasks | Medium | High |
| **Plan-and-Execute** | Complex adaptive workflows | High | Very High |

This pattern is particularly useful for tasks that require multiple steps or the use of various tools to complete.

# Setup and Configuration

## Install Dependencies

Install the required packages for Microsoft Agent Framework and Azure services.

In [ ]:
# Install Microsoft Agent Framework and dependencies
# Uncomment the line below if you haven't installed the packages yet
# !pip install agent-framework azure-identity azure-ai-projects python-dotenv httpx

## Import Required Libraries

In [1]:
import asyncio
import os
import sys
import json
import logging
from typing import List, Dict, Any, Annotated, cast
from dotenv import load_dotenv

# Microsoft Agent Framework core components
from agent_framework import (
    ChatAgent,
    ChatMessage,
    TextContent,
    Role,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    handler,
    ai_function,
)

# Azure integrations
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

# Utility imports
from typing_extensions import Never

# Add parent directory to path for utility imports
parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import search utility
try:
    from utils.search_utils import url_search
except ImportError:
    print("Warning: search_utils not found. Web search functionality will be limited.")
    url_search = None

# Load environment variables
load_dotenv(override=True)

# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

if logger.handlers:
    logger.handlers.clear()

handler_obj = logging.StreamHandler(sys.stdout)
handler_obj.setLevel(logging.INFO)
formatter = logging.Formatter('%(levelname)s: %(message)s')
handler_obj.setFormatter(formatter)
logger.addHandler(handler_obj)
logger.propagate = False

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Configure Environment Variables

Load Azure OpenAI credentials from environment variables.

In [2]:
# Get Azure OpenAI configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")

# Optional: Google Search API for web search
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Validate required environment variables
if not AZURE_OPENAI_ENDPOINT or not AZURE_OPENAI_API_KEY or not AZURE_OPENAI_DEPLOYMENT:
    raise ValueError(
        "Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY, and "
        "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME in .env file"
    )

print(f"✓ Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"✓ Deployment Name: {AZURE_OPENAI_DEPLOYMENT}")

# Check if Google Search API keys are available
if not GOOGLE_API_KEY or not GOOGLE_CSE_ID:
    print("⚠️  Warning: GOOGLE_API_KEY or GOOGLE_CSE_ID not set. Web search functionality will be limited.")
else:
    print("✓ Google Search API configured")

✓ Azure OpenAI Endpoint: https://hyo-ai-foundry-pjt1-resource.openai.azure.com/
✓ Deployment Name: gpt-4.1-mini
✓ Google Search API configured


## Initialize Azure OpenAI Chat Client

Create the chat client that will be used by our agents and workflows.

In [3]:
# Create Azure OpenAI Chat Client
azure_openai_chat_client = AzureOpenAIChatClient(
    model_id=AZURE_OPENAI_DEPLOYMENT,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

print("✓ Azure OpenAI Chat Client initialized")

✓ Azure OpenAI Chat Client initialized


# Part 1: Creating Custom Tools (Functions)

Before building workflows, we need to define the tools that our executors can use. These are custom functions decorated with `@ai_function` that provide specific capabilities.

In Microsoft Agent Framework:
- **Tools** are Python functions decorated with `@ai_function`
- They can be used by ChatAgents directly or called within Executor handlers
- They provide reusable capabilities across different workflows

## Define Product Search Tools

We'll create three tools for product research:
1. **search_product**: Search for information about a product
2. **compare_products**: Compare features between two products
3. **recommend_product**: Provide recommendations based on preferences

In [4]:
# Tool 1: Search for product information
@ai_function(description="Searches for information about a product using web search")
def search_product(query: Annotated[str, "The product to search for"]) -> str:
    """
    Searches for information about a specified product using web search.
    
    Args:
        query: The product to search for.
        
    Returns:
        Information about the product.
    """
    logger.info(f"🔍 Searching for product: {query}")
    
    # Try web search if available
    if url_search:
        results = url_search(query=query, max_result=3, web_search_mode="bing")
        
        if results:
            product_info = f"Found information about {query} from web search:\n\n"
            
            for i, result in enumerate(results):
                title = result.get("title", "Untitled")
                link = result.get("link", "")
                snippet = result.get("snippet", result.get("text_content", ""))
                
                product_info += f"{i+1}. {title}\n"
                product_info += f"   URL: {link}\n"
                if snippet:
                    product_info += f"   Summary: {snippet}\n"
                product_info += "\n"
                
            return product_info
    
    # Fallback if web search is not available
    return f"Could not find specific information about '{query}'. Web search may not be configured."


# Tool 2: Compare two products
@ai_function(description="Compares features between two products")
def compare_products(
    product1: Annotated[str, "First product to compare"],
    product2: Annotated[str, "Second product to compare"]
) -> str:
    """
    Compares features between two products using web search.
    
    Args:
        product1: First product to compare.
        product2: Second product to compare.
        
    Returns:
        Comparison between the products.
    """
    logger.info(f"⚖️  Comparing products: {product1} vs {product2}")
    
    if url_search:
        # Search for each product
        product1_results = url_search(query=product1, max_result=3, web_search_mode="bing")
        product2_results = url_search(query=product2, max_result=3, web_search_mode="bing")

        if product1_results and product2_results:
            comparison = f"Comparison between {product1} and {product2}:\n\n"
            
            # First product info
            comparison += f"## {product1.capitalize()} Information:\n"
            for i, result in enumerate(product1_results[:2]):
                title = result.get("title", "Untitled")
                snippet = result.get("snippet", result.get("text_content", "No description"))
                comparison += f"{i+1}. {title}\n   {snippet}\n\n"
            
            # Second product info
            comparison += f"## {product2.capitalize()} Information:\n"
            for i, result in enumerate(product2_results[:2]):
                title = result.get("title", "Untitled")
                snippet = result.get("snippet", result.get("text_content", "No description"))
                comparison += f"{i+1}. {title}\n   {snippet}\n\n"
            
            return comparison
    
    return f"Comparison data not available. Try with more specific product names."


# Tool 3: Recommend products based on preferences
@ai_function(description="Provides product recommendations based on user preferences")
def recommend_product(preferences: Annotated[str, "User preferences for recommendations"]) -> str:
    """
    Recommends products based on user preferences using web search.
    
    Args:
        preferences: User preferences for product recommendations.
        
    Returns:
        Product recommendations.
    """
    logger.info(f"💡 Generating recommendations for: {preferences}")
    
    if url_search:
        # Create a search query based on user preferences
        search_query = f"best products for {preferences}"
        results = url_search(query=search_query, max_result=3, web_search_mode="bing")
        
        if results:
            recommendations = f"Based on your preferences '{preferences}', here are some recommendations:\n\n"
            
            for i, result in enumerate(results):
                title = result.get("title", "Untitled")
                snippet = result.get("snippet", result.get("text_content", "No description"))
                
                recommendations += f"{i+1}. {title}\n"
                recommendations += f"   {snippet}\n\n"
                
            return recommendations
    
    return f"Recommendations for '{preferences}' are based on general knowledge. More specific criteria would help."

print("✓ Custom tools (functions) defined successfully")

✓ Custom tools (functions) defined successfully


## Test Tools with ChatAgent

Before building workflows, let's test our tools with a simple ChatAgent to ensure they work correctly.

In [5]:
# Create a ChatAgent with our custom tools
test_agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful product specialist. Use the available tools to answer user questions.",
    name="ProductAssistant",
    tools=[search_product, compare_products, recommend_product]
)

# Test the agent with a simple query
async def test_tools_with_agent():
    print("=== Testing Tools with ChatAgent ===\n")
    
    query = "I need a recommendation for a laptop with good battery life for a college student."
    print(f"User: {query}\n")
    print("Agent: ", end="", flush=True)
    
    async for update in test_agent.run_stream(query):
        if update.text:
            print(update.text, end="", flush=True)
    print("\n")

await test_tools_with_agent()

=== Testing Tools with ChatAgent ===

User: I need a recommendation for a laptop with good battery life for a college student.

Agent: INFO: 💡 Generating recommendations for: good battery life, suitable for college student
INFO: 💡 Generating recommendations for: good battery life, suitable for college student
For a college student looking for a laptop with good battery life,For a college student looking for a laptop with good battery life, here are some top recommendations based on recent reviews:

1. L here are some top recommendations based on recent reviews:

1. Laptops featured in "aptops featured in "Best Laptop For College: Top 7 Picks For Students 2025"
2.Best Laptop For College: Top 7 Picks For Students 2025"
2. Laptops listed in "The best laptops for battery life in  Laptops listed in "The best laptops for battery life in 2025: our top picks"
3. Laptops2025: our top picks"
3. Laptops with with the best battery life tested by PCMag in 2025 the best battery life tested by PCMag 

# Part 2: Sequential Workflow Pattern

Now let's implement a **sequential workflow** where tasks are executed one after another. This is useful when each step depends on the previous step's output.

## Sequential Workflow Architecture

```
Input → Executor 1 → Executor 2 → Executor 3 → Output
```

Based on `sequential_executors.py`, we'll create:
1. A **SearchExecutor** that searches for product information
2. A **SummarizerExecutor** that summarizes the search results
3. Connect them sequentially using WorkflowBuilder

## Define Sequential Executors

Let's create executors for a sequential workflow.

In [6]:
class SearchExecutor(Executor):
    """
    Executor that searches for product information using custom tools.
    
    This executor receives a user query, performs a search, and forwards
    the results to the next executor in the workflow.
    """
    
    def __init__(self, id: str):
        super().__init__(id=id)
    
    @handler
    async def search(self, query: str, ctx: WorkflowContext[str]) -> None:
        """
        Search for product information and forward results.
        
        Args:
            query: User's search query
            ctx: Workflow context for sending messages to next executor
        """
        logger.info(f"SearchExecutor: Received query '{query}'")
        
        # Use our custom search tool
        search_results = search_product(query)
        
        logger.info(f"SearchExecutor: Found {len(search_results)} characters of results")
        
        # Forward results to next executor
        await ctx.send_message(search_results)


class SummarizerExecutor(Executor):
    """
    Executor that summarizes search results using an LLM.
    
    This executor receives search results, uses an LLM to create a concise
    summary, and yields the final output.
    """
    
    def __init__(self, id: str, chat_client: AzureOpenAIChatClient):
        super().__init__(id=id)
        self._chat_client = chat_client
    
    @handler
    async def summarize(self, search_results: str, ctx: WorkflowContext[Never, str]) -> None:
        """
        Summarize search results and yield final output.
        
        Args:
            search_results: Results from SearchExecutor
            ctx: Workflow context for yielding final output
        """
        logger.info("SummarizerExecutor: Summarizing search results")
        
        # Create messages for LLM
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text="You are a helpful product specialist. Summarize the product information concisely, focusing on key features and benefits."
            ),
            ChatMessage(
                role=Role.USER,
                text=f"Summarize this product information:\n\n{search_results}"
            )
        ]
        
        # Get summary from LLM
        response = await self._chat_client.get_response(messages=messages)
        summary = response.messages[-1].text
        
        logger.info("SummarizerExecutor: Summary complete")
        
        # Yield final output
        await ctx.yield_output(summary)

print("✓ Sequential executors defined")

✓ Sequential executors defined


## Build and Run Sequential Workflow

Now let's build the sequential workflow and test it.

In [7]:
async def run_sequential_workflow():
    """
    Demonstrates a sequential workflow: Search → Summarize
    
    This workflow takes a user query, searches for product information,
    then summarizes the results.
    """
    print("=== Sequential Workflow: Search → Summarize ===\n")
    
    # Create executor instances
    search_executor = SearchExecutor(id="search_executor")
    summarizer_executor = SummarizerExecutor(id="summarizer_executor", chat_client=azure_openai_chat_client)
    
    # Build the workflow
    workflow = (
        WorkflowBuilder()
        .add_edge(search_executor, summarizer_executor)  # Sequential: search → summarize
        .set_start_executor(search_executor)
        .build()
    )
    
    # Run the workflow with a user query
    user_query = "gaming laptop with RTX 4070"
    print(f"User Query: {user_query}\n")
    print("Workflow Progress:")
    print("-" * 50)
    
    # Stream events to see the workflow in action
    outputs: list[str] = []
    async for event in workflow.run_stream(user_query):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            outputs.append(cast(str, event.data))
    
    print("-" * 50)
    print("\nFinal Summary:")
    if outputs:
        print(outputs[0])
    print()

await run_sequential_workflow()

=== Sequential Workflow: Search → Summarize ===

User Query: gaming laptop with RTX 4070

Workflow Progress:
--------------------------------------------------
Event: WorkflowStartedEvent(origin=WorkflowEventSource.FRAMEWORK, data=None)
Event: WorkflowStatusEvent(state=WorkflowRunState.IN_PROGRESS, data=None, origin=WorkflowEventSource.FRAMEWORK)
INFO: SearchExecutor: Received query 'gaming laptop with RTX 4070'
INFO: 🔍 Searching for product: gaming laptop with RTX 4070
INFO: 🔍 Searching for product: gaming laptop with RTX 4070
INFO: SearchExecutor: Found 852 characters of results
Event: ExecutorInvokedEvent(executor_id=search_executor, data=None)
Event: ExecutorCompletedEvent(executor_id=search_executor, data=None)
INFO: SummarizerExecutor: Summarizing search results
INFO: SearchExecutor: Found 852 characters of results
Event: ExecutorInvokedEvent(executor_id=search_executor, data=None)
Event: ExecutorCompletedEvent(executor_id=search_executor, data=None)
INFO: SummarizerExecutor: Sum

# Part 3: Parallel Workflow Pattern (Fan-out / Fan-in)

Now let's implement a **parallel workflow** where multiple tasks execute concurrently and their results are aggregated.

## Parallel Workflow Architecture

```
                    ┌─→ Executor 1 ─┐
Input → Dispatcher ─┼─→ Executor 2 ─┼→ Aggregator → Output
                    └─→ Executor 3 ─┘
```

Based on `aggregate_results_of_different_types.py`, we'll create:
1. A **Dispatcher** that broadcasts input to multiple executors
2. Multiple **task executors** that work in parallel
3. An **Aggregator** that collects and combines results

## Define Parallel Workflow Executors

Let's create executors for a parallel workflow that handles product research concurrently.

In [8]:
class ProductDispatcher(Executor):
    """
    Dispatcher that broadcasts product query to multiple research executors.
    """
    
    @handler
    async def dispatch(self, query: str, ctx: WorkflowContext[str]) -> None:
        """
        Dispatch the query to multiple executors for parallel processing.
        
        Args:
            query: User's product query
            ctx: Workflow context for broadcasting messages
        """
        if not query:
            raise RuntimeError("Query must be a valid string.")
        
        logger.info(f"ProductDispatcher: Broadcasting query '{query}' to all executors")
        await ctx.send_message(query)


class FeatureResearchExecutor(Executor):
    """
    Executor that researches product features.
    """
    
    def __init__(self, id: str, chat_client: AzureOpenAIChatClient):
        super().__init__(id=id)
        self._chat_client = chat_client
    
    @handler
    async def research_features(self, query: str, ctx: WorkflowContext[str]) -> None:
        """
        Research and extract key features of the product.
        
        Args:
            query: Product to research
            ctx: Workflow context for sending results
        """
        logger.info(f"FeatureResearchExecutor: Researching features for '{query}'")
        
        # Search for product
        search_results = search_product(query)
        
        # Extract features using LLM
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text="You are a product analyst. Extract and list the key technical features and specifications."
            ),
            ChatMessage(
                role=Role.USER,
                text=f"Extract key features from this product information:\n\n{search_results}"
            )
        ]
        
        response = await self._chat_client.get_response(messages=messages)
        features = response.messages[-1].text
        
        logger.info("FeatureResearchExecutor: Features extracted")
        await ctx.send_message(f"**Features:**\n{features}")


class PriceResearchExecutor(Executor):
    """
    Executor that researches product pricing information.
    """
    
    def __init__(self, id: str, chat_client: AzureOpenAIChatClient):
        super().__init__(id=id)
        self._chat_client = chat_client
    
    @handler
    async def research_price(self, query: str, ctx: WorkflowContext[str]) -> None:
        """
        Research pricing information for the product.
        
        Args:
            query: Product to research
            ctx: Workflow context for sending results
        """
        logger.info(f"PriceResearchExecutor: Researching price for '{query}'")
        
        # Search for product pricing
        search_results = search_product(f"{query} price")
        
        # Extract pricing using LLM
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text="You are a product analyst. Extract pricing information and value analysis."
            ),
            ChatMessage(
                role=Role.USER,
                text=f"Extract pricing information from this:\n\n{search_results}"
            )
        ]
        
        response = await self._chat_client.get_response(messages=messages)
        price_info = response.messages[-1].text
        
        logger.info("PriceResearchExecutor: Pricing information extracted")
        await ctx.send_message(f"**Pricing:**\n{price_info}")


class ReviewResearchExecutor(Executor):
    """
    Executor that researches product reviews and ratings.
    """
    
    def __init__(self, id: str, chat_client: AzureOpenAIChatClient):
        super().__init__(id=id)
        self._chat_client = chat_client
    
    @handler
    async def research_reviews(self, query: str, ctx: WorkflowContext[str]) -> None:
        """
        Research user reviews and ratings for the product.
        
        Args:
            query: Product to research
            ctx: Workflow context for sending results
        """
        logger.info(f"ReviewResearchExecutor: Researching reviews for '{query}'")
        
        # Search for product reviews
        search_results = search_product(f"{query} reviews")
        
        # Extract review summary using LLM
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text="You are a product analyst. Summarize user reviews, ratings, and common feedback."
            ),
            ChatMessage(
                role=Role.USER,
                text=f"Summarize user reviews from this information:\n\n{search_results}"
            )
        ]
        
        response = await self._chat_client.get_response(messages=messages)
        reviews = response.messages[-1].text
        
        logger.info("ReviewResearchExecutor: Reviews summarized")
        await ctx.send_message(f"**Reviews:**\n{reviews}")


class ResearchAggregator(Executor):
    """
    Aggregator that combines results from multiple research executors.
    """
    
    def __init__(self, id: str, chat_client: AzureOpenAIChatClient):
        super().__init__(id=id)
        self._chat_client = chat_client
    
    @handler
    async def aggregate(self, results: list[str], ctx: WorkflowContext[Never, str]) -> None:
        """
        Aggregate research results from all executors into a comprehensive report.
        
        Args:
            results: List of results from upstream executors
            ctx: Workflow context for yielding final output
        """
        logger.info(f"ResearchAggregator: Aggregating {len(results)} results")
        
        # Combine all results
        combined_results = "\n\n".join(results)
        
        # Create a comprehensive report using LLM
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text="You are a product specialist. Create a comprehensive, well-structured product report."
            ),
            ChatMessage(
                role=Role.USER,
                text=f"Create a comprehensive product report from these research results:\n\n{combined_results}"
            )
        ]
        
        response = await self._chat_client.get_response(messages=messages)
        final_report = response.messages[-1].text
        
        logger.info("ResearchAggregator: Final report created")
        await ctx.yield_output(final_report)

print("✓ Parallel workflow executors defined")

✓ Parallel workflow executors defined


## Build and Run Parallel Workflow

Now let's build the parallel workflow using fan-out and fan-in patterns.

In [9]:
async def run_parallel_workflow():
    """
    Demonstrates a parallel workflow with fan-out and fan-in pattern.
    
    This workflow dispatches a query to three executors that work in parallel:
    - FeatureResearchExecutor: Extracts product features
    - PriceResearchExecutor: Researches pricing
    - ReviewResearchExecutor: Summarizes reviews
    
    Results are then aggregated into a comprehensive report.
    """
    print("=== Parallel Workflow: Fan-out / Fan-in Pattern ===\n")
    
    # Create executor instances
    dispatcher = ProductDispatcher(id="dispatcher")
    feature_researcher = FeatureResearchExecutor(id="feature_researcher", chat_client=azure_openai_chat_client)
    price_researcher = PriceResearchExecutor(id="price_researcher", chat_client=azure_openai_chat_client)
    review_researcher = ReviewResearchExecutor(id="review_researcher", chat_client=azure_openai_chat_client)
    aggregator = ResearchAggregator(id="aggregator", chat_client=azure_openai_chat_client)
    
    # Build the parallel workflow
    workflow = (
        WorkflowBuilder()
        .set_start_executor(dispatcher)
        .add_fan_out_edges(dispatcher, [feature_researcher, price_researcher, review_researcher])
        .add_fan_in_edges([feature_researcher, price_researcher, review_researcher], aggregator)
        .build()
    )
    
    # Run the workflow with a product query
    user_query = "Samsung Galaxy S24 Ultra"
    print(f"User Query: {user_query}\n")
    print("Workflow Progress:")
    print("-" * 50)
    
    # Stream events to see parallel execution
    output: str | None = None
    async for event in workflow.run_stream(user_query):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            output = event.data
    
    print("-" * 50)
    print("\nComprehensive Product Report:")
    if output:
        print(output)
    print()

await run_parallel_workflow()

=== Parallel Workflow: Fan-out / Fan-in Pattern ===

User Query: Samsung Galaxy S24 Ultra

Workflow Progress:
--------------------------------------------------
Event: WorkflowStartedEvent(origin=WorkflowEventSource.FRAMEWORK, data=None)
Event: WorkflowStatusEvent(state=WorkflowRunState.IN_PROGRESS, data=None, origin=WorkflowEventSource.FRAMEWORK)
INFO: ProductDispatcher: Broadcasting query 'Samsung Galaxy S24 Ultra' to all executors
Event: ExecutorInvokedEvent(executor_id=dispatcher, data=None)
Event: ExecutorCompletedEvent(executor_id=dispatcher, data=None)
INFO: FeatureResearchExecutor: Researching features for 'Samsung Galaxy S24 Ultra'
INFO: 🔍 Searching for product: Samsung Galaxy S24 Ultra
Event: ExecutorInvokedEvent(executor_id=dispatcher, data=None)
Event: ExecutorCompletedEvent(executor_id=dispatcher, data=None)
INFO: FeatureResearchExecutor: Researching features for 'Samsung Galaxy S24 Ultra'
INFO: 🔍 Searching for product: Samsung Galaxy S24 Ultra
INFO: PriceResearchExecutor:

# Part 4: Plan and Execute Pattern with Workflow as Agent

Now let's implement the full **Plan and Execute pattern** by combining multiple workflows and wrapping them as an agent. This allows the workflow to be used just like a regular ChatAgent.

## Workflow as Agent Architecture

Based on `workflow_as_agent_reflection_pattern.py`, we can:
1. Build complex workflows with multiple executors
2. Wrap the workflow as an agent using `.as_agent()`
3. Interact with it using natural language queries

This is the most flexible pattern for handling complex, multi-step tasks.

## Define Plan and Execute Workflow

Let's create a workflow that can handle various product research tasks intelligently.

In [13]:
class QueryAnalyzerExecutor(Executor):
    """
    Executor that analyzes user queries and determines the appropriate action.
    
    This executor acts as a router, deciding whether to:
    - Search for a single product
    - Compare multiple products
    - Provide recommendations
    """
    
    def __init__(self, id: str, chat_client: AzureOpenAIChatClient):
        super().__init__(id=id)
        self._chat_client = chat_client
    
    @handler
    async def analyze_query(self, messages: list[ChatMessage], ctx: WorkflowContext[Dict[str, Any]]) -> None:
        """
        Analyze the user query and determine the action to take.
        
        Args:
            messages: List of chat messages from the user
            ctx: Workflow context for sending structured task information
        """
        # Extract the query from the last user message
        query = messages[-1].text if messages else ""
        
        logger.info(f"QueryAnalyzerExecutor: Analyzing query '{query}'")
        
        # Use LLM to analyze the query
        analysis_messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text="""You are a query analyzer. Determine the user's intent and respond in JSON format:
{
  "intent": "search" | "compare" | "recommend",
  "products": ["product1", "product2"],
  "preferences": "user preferences if recommending",
  "original_query": "original user query"
}

Examples:
- "Tell me about iPhone 15" → {"intent": "search", "products": ["iPhone 15"], "original_query": "..."}
- "Compare iPhone 15 vs Samsung S24" → {"intent": "compare", "products": ["iPhone 15", "Samsung S24"], "original_query": "..."}
- "Recommend a laptop for gaming" → {"intent": "recommend", "preferences": "gaming", "original_query": "..."}
"""
            ),
            ChatMessage(
                role=Role.USER,
                text=f"Analyze this query: {query}"
            )
        ]
        
        response = await self._chat_client.get_response(messages=analysis_messages)
        analysis_text = response.messages[-1].text
        
        # Parse JSON response
        try:
            # Extract JSON from markdown code block if present
            if "```json" in analysis_text:
                analysis_text = analysis_text.split("```json")[1].split("```")[0].strip()
            elif "```" in analysis_text:
                analysis_text = analysis_text.split("```")[1].split("```")[0].strip()
            
            analysis = json.loads(analysis_text)
            logger.info(f"QueryAnalyzerExecutor: Intent determined as '{analysis.get('intent')}'")
            await ctx.send_message(analysis)
        except Exception as e:
            logger.error(f"QueryAnalyzerExecutor: Failed to parse analysis: {e}")
            # Fallback to simple search
            await ctx.send_message({
                "intent": "search",
                "products": [query],
                "original_query": query
            })


class TaskExecutor(Executor):
    """
    Executor that performs the actual task based on query analysis.
    
    This executor uses our custom tools to execute the appropriate action.
    """
    
    def __init__(self, id: str, chat_client: AzureOpenAIChatClient):
        super().__init__(id=id)
        self._chat_client = chat_client
    
    @handler
    async def execute_task(self, task_info: Dict[str, Any], ctx: WorkflowContext[Never, str]) -> None:
        """
        Execute the task based on the analyzed intent.
        
        Args:
            task_info: Task information from QueryAnalyzerExecutor
            ctx: Workflow context for yielding final output
        """
        intent = task_info.get("intent", "search")
        products = task_info.get("products", [])
        preferences = task_info.get("preferences", "")
        original_query = task_info.get("original_query", "")
        
        logger.info(f"TaskExecutor: Executing {intent} task")
        
        result = ""
        
        if intent == "search" and products:
            # Search for single product
            result = search_product(products[0])
        
        elif intent == "compare" and len(products) >= 2:
            # Compare two products
            result = compare_products(products[0], products[1])
        
        elif intent == "recommend":
            # Provide recommendations
            result = recommend_product(preferences or original_query)
        
        else:
            result = f"Unable to process request: {original_query}"
        
        # Enhance the result with LLM
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text="You are a helpful product specialist. Provide a clear, informative response based on the information gathered."
            ),
            ChatMessage(
                role=Role.USER,
                text=f"User asked: {original_query}\n\nInformation gathered:\n{result}\n\nProvide a helpful response."
            )
        ]
        
        response = await self._chat_client.get_response(messages=messages)
        final_response = response.messages[-1].text
        
        logger.info("TaskExecutor: Task completed")
        await ctx.yield_output(final_response)

print("✓ Plan and execute executors defined")

✓ Plan and execute executors defined


## Build and Run Plan-and-Execute Workflow as Agent

Now let's build the workflow and wrap it as an agent that can handle various queries intelligently.

In [14]:
async def run_plan_and_execute_workflow():
    """
    Demonstrates the Plan and Execute pattern using Workflow as Agent.
    
    This workflow:
    1. Analyzes the user query to determine intent
    2. Executes the appropriate task (search, compare, or recommend)
    3. Returns a comprehensive response
    
    The workflow is wrapped as an agent, so it can be used just like ChatAgent.
    """
    print("=== Plan and Execute Pattern: Workflow as Agent ===\n")
    
    # Create executor instances
    query_analyzer = QueryAnalyzerExecutor(id="query_analyzer", chat_client=azure_openai_chat_client)
    task_executor = TaskExecutor(id="task_executor", chat_client=azure_openai_chat_client)
    
    # Build the workflow
    workflow = (
        WorkflowBuilder()
        .add_edge(query_analyzer, task_executor)  # Analyzer → Executor
        .set_start_executor(query_analyzer)
        .build()
    )
    
    # Wrap workflow as an agent
    agent = workflow.as_agent()
    
    print("Workflow wrapped as agent successfully!")
    print("This agent can handle search, comparison, and recommendation queries.\n")
    
    # Test with various queries
    test_queries = [
        #"What are the features of MacBook Pro M3?",
        "Compare iPhone 15 Pro and Samsung Galaxy S24",
        "Recommend a budget-friendly laptop for students"
    ]
    
    for query in test_queries:
        print(f"User: {query}")
        print("Agent: ", end="", flush=True)
        
        # Use the workflow as an agent
        async for event in agent.run_stream(query):
            if hasattr(event, 'text') and event.text:
                print(event.text, end="", flush=True)
        
        print("\n" + "=" * 80 + "\n")
    
    print("Plan and Execute workflow completed!")

await run_plan_and_execute_workflow()

=== Plan and Execute Pattern: Workflow as Agent ===

Workflow wrapped as agent successfully!
This agent can handle search, comparison, and recommendation queries.

User: Compare iPhone 15 Pro and Samsung Galaxy S24
Agent: INFO: QueryAnalyzerExecutor: Analyzing query 'Compare iPhone 15 Pro and Samsung Galaxy S24'
INFO: QueryAnalyzerExecutor: Analyzing query 'Compare iPhone 15 Pro and Samsung Galaxy S24'
INFO: QueryAnalyzerExecutor: Intent determined as 'compare'
INFO: TaskExecutor: Executing compare task
INFO: ⚖️  Comparing products: iPhone 15 Pro vs Samsung Galaxy S24
INFO: QueryAnalyzerExecutor: Intent determined as 'compare'
INFO: TaskExecutor: Executing compare task
INFO: ⚖️  Comparing products: iPhone 15 Pro vs Samsung Galaxy S24
INFO: TaskExecutor: Task completed
INFO: TaskExecutor: Task completed


User: Recommend a budget-friendly laptop for students
Agent: INFO: QueryAnalyzerExecutor: Analyzing query 'Recommend a budget-friendly laptop for students'


User: Recommend a budget-f

# Summary and Key Takeaways

## What We Learned

In this notebook, we explored the **Plan and Execute pattern** using Microsoft Agent Framework:

### 1. **Sequential Workflows**
- Tasks execute one after another
- Each step builds on the previous step's output
- Best for linear, dependent tasks
- Example: Search → Summarize

### 2. **Parallel Workflows (Fan-out/Fan-in)**
- Multiple tasks execute concurrently
- Results are aggregated into a final output
- Best for independent tasks that can run simultaneously
- Example: Feature Research ∥ Price Research ∥ Review Research → Aggregate

### 3. **Plan and Execute (Workflow as Agent)**
- Intelligent query analysis and routing
- Dynamic task execution based on intent
- Can be used like a regular ChatAgent
- Example: Analyze Query → Execute Appropriate Task

## Key Microsoft Agent Framework Concepts

| Concept | Purpose | Example |
|---------|---------|---------|
| **`@ai_function`** | Define reusable tools | `search_product()`, `compare_products()` |
| **`Executor`** | Define workflow steps | `SearchExecutor`, `QueryAnalyzerExecutor` |
| **`@handler`** | Handle messages in executors | `async def search(...)` |
| **`WorkflowBuilder`** | Build workflow graphs | `.add_edge()`, `.add_fan_out_edges()` |
| **`WorkflowContext`** | Pass data between executors | `ctx.send_message()`, `ctx.yield_output()` |
| **`.as_agent()`** | Wrap workflow as agent | `workflow.as_agent()` |

## Practical Applications

The Plan and Execute pattern can be applied to various real-world scenarios:

- **Customer Support**: Multi-step troubleshooting workflows
- **Research Assistance**: Breaking down complex research questions
- **Task Automation**: Combining multiple API calls and data transformations
- **Product Recommendations**: Gathering preferences and matching products
- **Data Analysis**: Sequential data processing pipelines

## Next Steps

To extend this pattern, consider:

1. **Add error handling** to retry failed steps
2. **Implement dynamic replanning** based on execution results
3. **Incorporate user feedback** between steps (Human-in-the-loop)
4. **Add more specialized tools** for specific domains
5. **Implement conditional branching** in workflows
6. **Add observability** with Azure Monitor and Application Insights

## Additional Resources

- [Microsoft Agent Framework Documentation](https://learn.microsoft.com/en-us/agent-framework/)
- [Agent Framework GitHub Repository](https://github.com/microsoft/agent-framework)
- [Agent Framework Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples)
- [Workflow Patterns Guide](https://microsoft.github.io/agent-framework/workflows/)

## Comparison with Other Patterns

### Plan and Execute vs. Other Agentic Patterns

| Pattern | Complexity | Best For | Limitations |
|---------|-----------|----------|-------------|
| **Plan and Execute** | High | Complex multi-step tasks | Requires good task decomposition |
| **Reflection** | Medium | Iterative improvement | Can be slower due to iterations |
| **Multi-Agent** | Very High | Specialized agent collaboration | Complex coordination, slow latency |
| **RAG (Retrieval)** | Low-Medium | Knowledge-intensive tasks | Limited to available documents |

### When to Use Plan and Execute

✅ **Use when:**
- Tasks require multiple distinct steps
- Steps can be executed independently (parallel) or sequentially
- You need transparency in task decomposition
- Workflows need to be reusable and composable

❌ **Don't use when:**
- Task is simple and single-step
- Real-time response is critical (workflows add overhead)
- Task requires continuous user interaction
- The workflow structure is highly dynamic and unpredictable